# Tender2Project
The following Kaggle notebook exploits the long context window of Gemini, in order to fulfill the following targets:
- Analyze the tender for a project.
- Analyze the information about possible products, provided from different companies.
- Find the best combination of products to build the project in the tender, identifying the most compliant company as well.

## Notebook structure
The notebook is composed by different parts, each one with a specific target:
- A tender for a project is parsed, such that its information is converted to text.
- Information scraped from the websites of different companies is loaded as text.
- All the text is forwarded to Gemini, whereas a system prompt and a user prompt are written to explain the purposed o Gemini.

## Theoretical aspects
The current way to use Gemini makes use of the following properties of a LLM (Large Language Model) like Gemini:
|  **LLM property** | **Where it is used** | **How it is used** |
|:-----------------:|:--------------------:|:------------------:|
|     Reasoning     |          TBD         |         TBD        |
|       Memory      |          TBD         |         TBD        |
| Chain of Thoughts |          TBD         |         TBD        |
|                   |                      |                    |

Clean the working directory

In [1]:
! rm -r /kaggle/working/*

rm: cannot remove '/kaggle/working/*': No such file or directory


In [2]:
from IPython.display import Markdown

# Load the tenders

Locate the tenders - in PDF format.

In [3]:
# fetch the script to download content from GitHub
!wget https://raw.githubusercontent.com/gabripo/kaggle-gemini-long-context/refs/heads/main/github_downloader.py -P /kaggle/working/scripts

# add downloaded script to the Python path
import sys
sys.path.append('/kaggle/working/scripts')

--2024-11-23 10:49:36--  https://raw.githubusercontent.com/gabripo/kaggle-gemini-long-context/refs/heads/main/github_downloader.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1722 (1.7K) [text/plain]
Saving to: '/kaggle/working/scripts/github_downloader.py'

github_downloader.p 100%[===================>]   1.68K  --.-KB/s    in 0s      

2024-11-23 10:49:36 (32.6 MB/s) - '/kaggle/working/scripts/github_downloader.py' saved [1722/1722]



In [4]:
import github_downloader

github_downloader.download_files_from_github_repo(folderName="tenders", saveFolder="/kaggle/working/tenders", extension="pdf")

/kaggle/working/tenders/tender_solar.pdf downloaded successfully.
/kaggle/working/tenders/tender_wind.pdf downloaded successfully.
All files downloaded.


In [5]:
import os

# if the PDF file is given as Kaggle Input (for example, manually uploaded), change the use_kaggle_input to True
use_kaggle_input_tender = False if os.path.exists('/kaggle/working/tenders') else True
if use_kaggle_input_tender:
    tenders_file_path = '/kaggle/input/tenders'
else:
    tenders_file_path = '/kaggle/working/tenders'

print(f"The folder {tenders_file_path} will be considered as containing the tenders")

The folder /kaggle/working/tenders will be considered as containing the tenders


Installing required Python packages to analyze the tender - in PDF format.

In [6]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.4 MB/s eta 0:00:00


Extract information from the tenders.
The output will be a text.

In [7]:
import os
from PyPDF2 import PdfReader

tenders = [t for t in os.listdir(tenders_file_path) if t.endswith(".pdf")]
tenders_info = {}
for tender in tenders:
    print(f"Reading the tender {tender} ...")
    reader = PdfReader(os.path.join(tenders_file_path, tender))

    tenders_info[tender] = {}
    tenders_info[tender]["name"] = tender
    tenders_info[tender]["content"] = "\n".join([page.extract_text() for page in reader.pages])

# information for each tender can be accessed by:
# tenders_info[tenders[0]]

Reading the tender tender_solar.pdf ...
Reading the tender tender_wind.pdf ...


# Fetch information about companies

## Overview
Information about interesting companies is obtained from their websites.

To generate data out of the companies' websites, we implemented a crawler.
The final output of the crawler is a JSON file, in which each field refers to a company: for each company, all the information of the websites is merged.

> To make things easier, the mentioned JSON file will be fetched from a Git repository where the crawling function has already been executed.

## Details about the crawling process:
- **Recursive scan**: after a webpage is scanned and its content is stored, eventual found sublinks are scanned, as well. A limit of the wepages to download is given as input.
- **Redundant information is deleted**: if some website content can be found multiple times in all the webpages of one company, then it is skipped. *Example*: undesired and redundant lines like "Contact Us" are removed, ensuring that the final content does not include unnecessary sentences.
- **Caching of already downloaded pages**: for each webpage, the content is stored in a JSON file, as well as the found sublinks. *Example*: after a run with a limit of N pages, other runs with less than N pages will use the stored files instead downloading data from internet; at the contrary, if the limit is increased to M > N pages, only M - N additional pages will be downloaded while the first N pages will be taken from the stored file.

## Load the results from the crawler's repo

In [8]:
github_downloader.download_files_from_github_repo(folderName="", saveFolder="/kaggle/working/companies_info", extension="json")

/kaggle/working/companies_info/companies_info.json downloaded successfully.
All files downloaded.


Locate the JSON file containing the companies' information.

In [9]:
import os

# if the JSON file is given as Kaggle Input (for example, manually uploaded), change the use_kaggle_input to True
use_kaggle_input_companies = False if os.path.exists('/kaggle/working/companies_info') else True
companies_json_name = 'companies_info.json'
if use_kaggle_input_companies:
    companies_info_file_path = os.path.join('/kaggle/input/companies-info', companies_json_name)
else:
    companies_info_file_path = os.path.join('/kaggle/working/companies_info', companies_json_name)

print(f"The file {companies_info_file_path} will be used for the information regarding the companies")

The file /kaggle/working/companies_info/companies_info.json will be used for the information regarding the companies


Define a small function to read the information about the companies - in JSON format.

In [10]:
import json

def read_json_info(jsonFilePath: str) -> dict:
    if os.path.exists(jsonFilePath):
        with open(jsonFilePath, "r") as f:
            data = json.load(f)
        return data
    else:
        return {}

Load the companies' information by using the defined function.

In [11]:
companies_info = read_json_info(companies_info_file_path)

# companies_info is a dictionary, where the key is the name of the company and the related value its information
# print(companies_info["SIEMENS"]) 

# Chat with Gemini

In [12]:
# API key got here: https://ai.google.dev/tutorials/setup

import google.generativeai as genai
from kaggle_secrets import UserSecretsClient


user_secrets = UserSecretsClient()
secret_key = user_secrets.get_secret("GEMINI_API_KEY")

genai.configure(api_key = secret_key)

model_name = 'gemini-1.5-flash-latest'
model = genai.GenerativeModel(model_name=model_name)

chat = model.start_chat()

model_info = genai.get_model(f"models/{model_name}")
print(f"{model_info.input_token_limit=}")
print(f"{model_info.output_token_limit=}")

model_info.input_token_limit=1000000
model_info.output_token_limit=8192


## Analyze all the tenders

In [13]:
download_tenders_json_from_repo = False # switch to False if willing to generate the json file within this notebook

if download_tenders_json_from_repo:
    # this helps reducing the Gemini Quota, since no queries will be performed for the tenders
    github_downloader.download_files_from_github_repo(folderName="tenders", saveFolder=tenders_file_path, extension="json")

In [14]:
tender_prompt_template = "The document you have is a tender, that contains technical requirements for a project. Summarize the technical requirements. The content of the document is: "
tender_prompts = []
for info in tenders_info.values():
    tender_prompts.append(f"You have a document called {info['name']} . " + tender_prompt_template + f"{info['content']}")

In [15]:
from time import sleep

responses = {}
tenders_json_file_path = os.path.join(tenders_file_path, 'tenders.json')
if os.path.exists(tenders_json_file_path):
    responses = read_json_info(tenders_json_file_path)
    print(f"Responses loaded from file {tenders_json_file_path}")
else:
    num_queries = 0
    for tender_prompt, tender_name in zip(tender_prompts, tenders):
        print(f"Generating response for tender {tender_name} ...")
        response = chat.send_message(tender_prompt)
        # print(response.text)
        responses[tender_name] = {'prompt': tender_prompt, 'answer': response.text}
        print(f"Response for tender {tender_name} generated.")

        num_queries += 1
        """
        if num_queries % 2 == 0:
            wait_time_seconds = 90
            print(f"Waiting {wait_time_seconds} before continuing, to not exceed Gemini's quota")
            sleep(wait_time_seconds)
        """

    with open(tenders_json_file_path, 'w') as f:
        json.dump(responses, f, ensure_ascii=True, indent=4)
    print(f"Responses stored into {tenders_json_file_path}")

print("Analysis of the tenders concluded!")

Generating response for tender tender_solar.pdf ...
Response for tender tender_solar.pdf generated.
Generating response for tender tender_wind.pdf ...
Response for tender tender_wind.pdf generated.
Responses stored into /kaggle/working/tenders/tenders.json
Analysis of the tenders concluded!


In [16]:
# example how to include the chat history here https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_chat.ipynb
# description of the Content class here https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/GenerativeModel.md
from google.generativeai.protos import Content, Part

history_chat = []
for idx_response, response in enumerate(responses.values()):
    query = Part()
    query.text = response['prompt']
    # TODO: consider different users? Example: for the tender number idx_response, use f"user_{idx_response}"
    history_chat.append(Content(role="user", parts=[query]))

    answer = Part()
    answer.text = response['answer']
    history_chat.append(Content(role="model", parts=[answer]))

## Define the system and user prompts

In [17]:
system_prompt = "You are an experienced technical sales manager. You are given the full portfolio for products and solutions of companies, through their website. Use this information to provide detailed technical answers based on the tender requirements which were given to you."

Markdown(system_prompt)

You are an experienced technical sales manager. You are given the full portfolio for products and solutions of companies, through their website. Use this information to provide detailed technical answers based on the tender requirements which were given to you.

In [18]:
user_prompt = f"""

1. Identify all the technical requirements in the tenders' information you have. This information is in the form of text I provided, then you do not need to read additional documents.

2. For company [SIEMENS] and [HITACHI], find the respective relevant products and solutions with respect to point 1. Do this one company at a time and store the results. This information is in the form of text I provided, then you do not need to read additional documents or access to websites.

3. Calculate an affinity score in percentage of company [SIEMENS] and [HITACHI] based on the match of results in point 2. Explain the way how you computed this percentage.

SIEMENS: {companies_info["SIEMENS"]}
HITACHI: {companies_info["HITACHI"]}

4. Return a quick technical summary of the tender.

5. Return the technical details of the company with the highest affinity score and its score. Focus on the technical specifications mentioning the compliances with the tender. Report also the URL of the source where you found the informations and mention if there is some non compliant requirements from tender.

6. Return also the other affinity scores alone of the other companies. 

"""

# Markdown(user_prompt)

## Test the history

In [19]:
chat_with_memory = model.start_chat(history=history_chat)
response = chat_with_memory.send_message("What are the tenders you have about?")

Markdown(response.text)

I've processed two tender documents:

* **`tender_solar.pdf`:** This tender outlines the technical requirements for the construction of a 200 kWp solar power plant and associated mini-grid in Barclayville.  It details specifications for solar panels, mounting structures, inverters, batteries, a diesel genset, electrical BOS components (cables, junction boxes), powerhouse buildings, monitoring systems, and grid infrastructure.  The tender also specifies requirements for manuals, warranties, spare parts, training, and after-sales service.

* **`tender_wind.pdf`:** This document is not a tender itself but rather a guide to designing effective tenders for wind energy projects.  It discusses various approaches to structuring tenders for both onshore and offshore wind power, analyzing different tender design options (e.g., centralized vs. decentralized site selection, technology-neutral vs. technology-specific, different payment mechanisms, etc.). The document reviews past tender experiences from several countries, highlighting successes and failures to provide guidance on best practices for future tender design.  The focus is on achieving both cost-effective and efficient deployment of wind energy while adhering to EU State aid guidelines.


In [20]:
response = chat_with_memory.send_message("Detailed explanation of the technical specifications of the tenders")

Markdown(response.text)

Let's break down the technical specifications of the two tenders in more detail:


**Tender_solar.pdf (Solar Power Plant):**

This tender specifies a 200 kWp solar PV mini-grid system. The technical requirements are extensive and cover various components:

* **1. Solar PV Array (200 kWp):**  The core of the system.  Specifications include:
    * **Capacity:** 200 kWp at Standard Test Conditions (STC).
    * **Technology:** Monocrystalline or polycrystalline silicon.
    * **Performance Warranty:** Minimum 25 years, guaranteeing at least 80% power output after 25 years and maximum 1% degradation annually.
    * **Safety and Quality Standards:**  Compliance with IEC 61215 (performance testing), IEC 61730 (safety), ISO 9001:2008 (quality management), and ISO 14001:2004 (environmental management).
    * **Module Specifications:**  Detailed requirements for peak power (20Wp - 350Wp), nominal power, rated voltage and current, open-circuit voltage, and short-circuit current are provided.  Clear labeling of the module's brand, model, and specifications is mandatory.

* **2. PV Mounting Structure (200 kWp):** The support structure for the solar panels.  Crucial specifications include:
    * **Material:** Hot-dip galvanized/anodized aluminum.
    * **Coating Thickness:** Minimum 120 microns.
    * **Wind Rating:**  Must withstand winds up to 150 km/h.
    * **Fasteners:** Stainless steel (SS 304).
    * **Ground Clearance:** Minimum 800mm.
    * **Spacing:** 4-meter gap between array structures and the boundary fence, 2-meter-wide pathways for maintenance access.

* **3. Inverter (140 kW):** Converts DC power from the solar panels to AC power for the grid. Key specifications are:
    * **Capacity:** 140 kW.
    * **Efficiency:** >95% typical conversion efficiency.
    * **No-load Loss:** <1% of rated power.
    * **Grid Synchronization:** Wide range of grid voltage and frequency parameters.
    * **Waveform:** Sinusoidal current modulation.
    * **Protection:** Comprehensive protection against overcurrent, synchronization loss, overtemperature, DC bus overvoltage, and power regulation during thermal overloading.
    * **Communication:**  Dedicated interfaces (Ethernet, Prefabs) for networking, remote control via modem or web server.
    * **Safety:** IP31 degree of protection.

* **4. Battery (400 kWh):** Energy storage for grid stabilization.  The requirements emphasize high power density over energy density and include:
    * **Technology:** Lithium-ion.
    * **Cycle Life:** Minimum 1,000 cycles at 90% depth of discharge (DOD).
    * **Self-Discharge:** Maximum 5% per month.
    * **Warranty:** 80% capacity retention after two years.
    * **Standards:**  Compliance with ISO 9001:2008, ISO 14001:2004, and IEC 61427-1 (or equivalent).
    * **Information:** Clear labeling of brand, model, rated voltage, capacity, and terminal polarity.

* **5. Diesel Genset (180 kW/225 kVA):** Backup power generation.  Specifications include:
    * **Capacity:** 180 kW (225 kVA).
    * **Engine Type:** Four-stroke, multi-cylinder, water-cooled, turbocharged with aftercooler.
    * **Starting:** Cold starting capability down to 0°C, direct start to run speed.
    * **Control System:** Smart starting control system with fuel ramping and frequency overshoot limitations.
    * **Monitoring:** Comprehensive monitoring, metering, and control with PC interface.
    * **Standards:** ISO 3046 / BS 5514 / IS 4722/1992.

* **6. Electrical BOS:**  Covers cables, junction boxes, and wiring.  Detailed specifications are provided for cable types, insulation, voltage ratings, installation methods, and protection.  Three types of junction boxes (array, sub-main, and main) are specified with requirements for waterproofing and surge protection.

* **7. Powerhouse Buildings and Parking:**  Detailed requirements for building size, location, and facilities.

* **8. Installation, Labor, Tools, and Equipment:**  Covers the transportation, storage, and installation of all equipment.

* **9. Monitoring System:** SCADA system with remote access, capable of monitoring various parameters (temperature, solar radiation, inverter output, energy production, etc.).  Must meet IEC 61724.

* **10. Warranty:** Minimum two-year warranty on the main system and battery (with specific capacity retention requirements).

* **11. Manuals:** Installation, maintenance, and troubleshooting manuals (in English) are required.

* **12. Substation (0.4/11 kV):** Detailed civil works and equipment specifications.

* **13-23. Mini-grid Components and SHS:** This section specifies underground and overhead MV/LV networks, transformers, customer connections (single and three-phase), and solar home systems (SHS).

**Tender_wind.pdf (Wind Energy Tender Design Guide):**

This document doesn't contain technical specifications for a specific wind farm project but rather provides guidelines for designing tenders for wind energy projects.  It outlines various options and considerations instead of fixed requirements.  However, the case studies within the document detail the technical specifications of wind tenders implemented in various countries, including:

* **Capacity:** The amount of power to be procured (in MW or MWh).
* **Support Mechanism:**  Mechanisms like Contracts for Difference (CfDs), feed-in premiums, or feed-in tariffs.
* **Price Determination:**  How the price will be determined (e.g., sealed bid, auction, negotiation).
* **Payment Arrangement:**  How the successful bidders will be paid (e.g., pay-as-bid, pay-as-clear).
* **Technology Specificity:** Whether the tender is technology-neutral or focuses on specific technologies.
* **Pre-qualification Criteria:**  Requirements for bidders (financial and technical capabilities).
* **Penalties:**  Consequences for failing to meet project milestones or contractual obligations.
* **Grid Connection:** Who is responsible for grid connection costs.
* **Planning Permissions:**  Requirements for securing necessary permits.

The case studies showcase the wide range of technical approaches and considerations involved in creating effective wind energy tenders, highlighting the importance of tailoring the tender design to specific market conditions and policy objectives.  They don't provide a single, universal set of technical specifications, but rather illustrate the variations used across different jurisdictions and their relative success or failure.


## Count the needed tokens

In [21]:
print(f"{model.count_tokens(history_chat)=}")
print(f"{model.count_tokens(system_prompt)=}")
print(f"{model.count_tokens(user_prompt)=}")

model.count_tokens(history_chat)=total_tokens: 20479

model.count_tokens(system_prompt)=total_tokens: 44

model.count_tokens(user_prompt)=total_tokens: 645267



## Generate the final response

In [22]:
# wait some time, until the Gemini's Quota comes back
sleep(60) # seconds

In [23]:
response = chat_with_memory.send_message(f"{user_prompt}")

Markdown(response.text)

ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [ ]:
# wait some time, until the Gemini's Quota comes back
sleep(60) # seconds

In [ ]:
print("Providing prompts based on the previous analysis of the tenders ...")
big_prompt = f"{system_prompt}\n\n{user_prompt}"
response = chat_with_memory.send_message(big_prompt)
# print(response.text)
responses['main_query'] = {'prompt': big_prompt, 'answer': response.text}

print("Response to the prompts is ready!")

In [ ]:
Markdown(responses['main_query']['answer'])